## Statistical Analysis by Structural Groups (RQ1 & RQ2)

**Structure groups**: Centralized (sequential_fanout, parallel_fanout), Structured (chain_with_branching, hierarchical_tree), Probabilistic (probabilistic_tree), Dense (complex_mesh).

**Steps:**
1. **Normality** — Shapiro–Wilk by topology; short summary + LaTeX paragraph (non-parametric justification).
2. **RQ1 Energy** — Kruskal–Wallis on **Energy (J)** only; structural contrasts (Dense vs Others, Centralized vs Structured, Seq vs Par, Prob vs Det). Energy per request / per success / per RPS are computed in the notebook but excluded from the paper output per discourse.
3. **Size interaction** — Topology × Size for energy and throughput (rank-based two-way).
4. **RQ2 Performance–Energy** — Spearman correlations per topology (throughput–energy, latency–energy, failure–energy).
5. **Efficiency** — energy_per_rps is computed but not included in the merged paper output.
6. **Output**: Only `tables/table3_4_5_statistics.tex` (single file to include in the paper).

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

cwd = Path('.').resolve()
# Find repo root by walking up to directory containing 5_results_data/run_table.csv
_run_table = Path('5_results_data') / 'run_table.csv'
_search = cwd
RUN_TABLE_PATH = None
for _ in range(6):
    candidate = _search / _run_table
    if candidate.exists():
        RUN_TABLE_PATH = candidate
        break
    _search = _search.parent
if RUN_TABLE_PATH is None:
    RUN_TABLE_PATH = cwd / _run_table
# Outputs go to 5_results_analysis/tables (one level up when running from notebooks/)
TABLES_DIR = (cwd.parent / 'tables') if cwd.name == 'notebooks' else (cwd / 'tables')
TABLES_DIR.mkdir(parents=True, exist_ok=True)

TOPOLOGY_TO_STRUCTURE = {
    'sequential_fanout': 'centralized',
    'parallel_fanout': 'centralized',
    'chain_with_branching': 'structured',
    'hierarchical_tree': 'structured',
    'probabilistic_tree': 'probabilistic',
    'complex_mesh': 'dense',
}
CORE_METRICS = ['throughput_rps', 'avg_latency_s', 'energy_kj', 'cpu_usage_avg']
OPTIONAL_METRICS = ['failure_rate', 'p95_latency_s']
METRIC_LABELS = {
    'throughput_rps': 'Throughput (RPS)',
    'avg_latency_s': 'Avg. response time (s)',
    'energy_kj': 'Energy (kJ)',
    'cpu_usage_avg': 'CPU utilization',
    'failure_rate': 'Failure rate',
    'p95_latency_s': 'P95 latency (s)',
}
ALPHA = 0.05
print("RUN_TABLE_PATH:", RUN_TABLE_PATH)

RUN_TABLE_PATH: /home/irena/Documents/Research Project/topology-scale-mubench-replication/5_results_data/run_table.csv


In [2]:
df = pd.read_csv(RUN_TABLE_PATH)
df = df[df['__done'] == 'DONE'].copy()
df['structure_group'] = df['topology'].map(TOPOLOGY_TO_STRUCTURE)
if 'failure_rate' in df.columns and df['failure_rate'].max() <= 1.5:
    df['failure_rate'] = df['failure_rate'] * 100
if 'cpu_usage_avg' in df.columns and df['cpu_usage_avg'].max() <= 1.5:
    df['cpu_usage_avg'] = df['cpu_usage_avg'] * 100
# Units for paper: energy in kJ, latency in s
df['energy_kj'] = df['energy'] / 1000
df['avg_latency_s'] = df['avg_latency_ms'] / 1000
df['p95_latency_s'] = df['p95_latency_ms'] / 1000
# Derived energy metrics (for RQ1 and efficiency; efficiency excluded from paper output)
df['energy_per_request'] = df['energy'] / df['request_count'].replace(0, np.nan)
safe_success = (df['request_count'] * (1 - df['failure_rate'] / 100)).replace(0, np.nan)
df['energy_per_success'] = df['energy'] / safe_success
df['energy_per_rps'] = df['energy'] / df['throughput_rps'].replace(0, np.nan)

metrics = CORE_METRICS + [m for m in OPTIONAL_METRICS if m in df.columns]
print(f"Loaded {len(df)} runs. Structure groups:\n{df['structure_group'].value_counts().to_dict()}")

Loaded 180 runs. Structure groups:
{'centralized': 60, 'structured': 60, 'probabilistic': 30, 'dense': 30}


### Step 1. Normality (Shapiro–Wilk) by topology

Per (metric, topology). Small summary table + short LaTeX paragraph: normality violated → non-parametric tests used.

In [3]:
NORM_METRICS = ['throughput_rps', 'avg_latency_s', 'p95_latency_s', 'energy_kj', 'failure_rate', 'cpu_usage_avg']
norm_rows = []
for col in NORM_METRICS:
    if col not in df.columns:
        continue
    for topo, grp in df.groupby('topology'):
        vals = grp[col].dropna()
        if len(vals) < 3:
            continue
        _, p = stats.shapiro(vals)
        norm_rows.append({'metric': col, 'topology': topo, 'p_value': p})
norm_df = pd.DataFrame(norm_rows)
violations = norm_df[norm_df['p_value'] < ALPHA]
majority_violated = violations.groupby('metric').size() > (norm_df.groupby('metric').size() / 2)
# norm_df.to_csv(...)  # only merged section written
print("Normality by topology (sample):")
print(norm_df.head(12).to_string())
print(f"\nMajority p < 0.05 (non-normal) per metric: {majority_violated.to_dict()}")
# Short LaTeX paragraph
norm_paragraph = (
    "Normality was assessed with Shapiro--Wilk tests per metric and topology. "
    "The majority of groups showed $p < 0.05$, indicating violation of normality assumptions; "
    "therefore non-parametric tests (Kruskal--Wallis, Mann--Whitney $U$, Spearman) are used throughout."
)
# Only merged section is written at the end
pass

Normality by topology (sample):
            metric              topology       p_value
0   throughput_rps  chain_with_branching  7.243727e-05
1   throughput_rps          complex_mesh  1.271340e-06
2   throughput_rps     hierarchical_tree  6.005541e-05
3   throughput_rps       parallel_fanout  1.546559e-04
4   throughput_rps    probabilistic_tree  9.380843e-05
5   throughput_rps     sequential_fanout  4.748467e-05
6    avg_latency_s  chain_with_branching  4.595411e-06
7    avg_latency_s          complex_mesh  9.001686e-07
8    avg_latency_s     hierarchical_tree  3.542815e-06
9    avg_latency_s       parallel_fanout  1.232518e-05
10   avg_latency_s    probabilistic_tree  1.398755e-04
11   avg_latency_s     sequential_fanout  2.522946e-06

Majority p < 0.05 (non-normal) per metric: {'avg_latency_s': True, 'cpu_usage_avg': True, 'energy_kj': True, 'failure_rate': True, 'p95_latency_s': True, 'throughput_rps': True}


### 2. Kruskal–Wallis (across four structure groups)

If p ≥ 0.05 for a metric, no pairwise testing for that metric.

In [4]:
kw_results = {}
for m in metrics:
    if m not in df.columns:
        continue
    groups = [g[m].dropna().values for _, g in df.groupby('structure_group')]
    if any(len(g) < 2 for g in groups):
        kw_results[m] = (np.nan, 1.0)
        continue
    h, p = stats.kruskal(*groups)
    kw_results[m] = (h, p)
    print(f"{m}: H={h:.3f}, p={p:.4f} {'*' if p < ALPHA else ''}")

throughput_rps: H=52.006, p=0.0000 *


avg_latency_s: H=51.620, p=0.0000 *
energy_kj: H=18.129, p=0.0004 *
cpu_usage_avg: H=38.656, p=0.0000 *
failure_rate: H=96.444, p=0.0000 *
p95_latency_s: H=54.321, p=0.0000 *


### 3. Pairwise Mann–Whitney U + Holm correction

Contrasts: Dense vs Others, Centralized vs Structured, Seq vs Par, Probabilistic vs Deterministic. Holm correction applied within each metric.

In [5]:
def dense_vs_others(df, metric):
    dense = df[df['structure_group'] == 'dense'][metric].dropna().values
    others = df[df['structure_group'].isin(['centralized', 'structured', 'probabilistic'])][metric].dropna().values
    if len(dense) < 2 or len(others) < 2:
        return np.nan
    _, p = stats.mannwhitneyu(dense, others, alternative='two-sided')
    return p

def centralized_vs_structured(df, metric):
    c = df[df['structure_group'] == 'centralized'][metric].dropna().values
    s = df[df['structure_group'] == 'structured'][metric].dropna().values
    if len(c) < 2 or len(s) < 2:
        return np.nan
    _, p = stats.mannwhitneyu(c, s, alternative='two-sided')
    return p

def seq_vs_par(df, metric):
    seq = df[df['topology'] == 'sequential_fanout'][metric].dropna().values
    par = df[df['topology'] == 'parallel_fanout'][metric].dropna().values
    if len(seq) < 2 or len(par) < 2:
        return np.nan
    _, p = stats.mannwhitneyu(seq, par, alternative='two-sided')
    return p

def probabilistic_vs_deterministic(df, metric):
    prob = df[df['structure_group'] == 'probabilistic'][metric].dropna().values
    det = df[df['structure_group'].isin(['centralized', 'structured'])][metric].dropna().values
    if len(prob) < 2 or len(det) < 2:
        return np.nan
    _, p = stats.mannwhitneyu(prob, det, alternative='two-sided')
    return p

def holm_correction(p_values):
    n = len(p_values)
    order = np.argsort(p_values)
    sorted_p = np.array(p_values)[order]
    corrected = np.minimum(sorted_p * (n - np.arange(n)), 1.0)
    out = np.empty_like(corrected)
    out[order] = corrected
    return out

In [6]:
pairwise_holm = {}
for m in metrics:
    if m not in df.columns:
        continue
    raw = [
        dense_vs_others(df, m),
        centralized_vs_structured(df, m),
        seq_vs_par(df, m),
        probabilistic_vs_deterministic(df, m),
    ]
    raw_f = [float(x) if not np.isnan(x) else 1.0 for x in raw]
    pairwise_holm[m] = holm_correction(raw_f)
print("Pairwise (Holm) computed for metrics where Kruskal–Wallis p < 0.05")

Pairwise (Holm) computed for metrics where Kruskal–Wallis p < 0.05


### 4. Build table and write outputs

One table: rows = metrics, columns = Dense vs Others | Centralized vs Structured | Seq vs Par | Probabilistic vs Deterministic. Cell = corrected p-value or "ns". Bold if p < 0.05.

In [7]:
def escape_latex(s):
    if not isinstance(s, str):
        return s
    return s.replace('_', r'\_').replace('&', r'\&').replace('%', r'\%')

col_headers = ['Dense vs Others', 'Centralized vs Structured', 'Seq vs Par', 'Probabilistic vs Deterministic']
rows_tex = []
interpretations = []

for metric in metrics:
    if metric not in df.columns:
        continue
    label = METRIC_LABELS.get(metric, metric.replace('_', ' '))
    ps = pairwise_holm.get(metric, [1.0] * 4)
    cells = []
    for j, p in enumerate(ps):
        p = ps[j] if j < len(ps) else 1.0
        if np.isnan(p):
            cells.append('—')
        else:
            s = f'{p:.3f}' if p >= 0.001 else f'{p:.2e}'
            cells.append(f'\\textbf{{{s}}}' if p < ALPHA else s)
    rows_tex.append([label] + cells)
    parts = []
    if metric in pairwise_holm:
        ps = pairwise_holm[metric]
        if not np.isnan(ps[0]) and ps[0] < ALPHA:
            parts.append('Dense vs Others significant → dense coordination cost.')
        if not np.isnan(ps[1]) and ps[1] < ALPHA:
            parts.append('Centralized vs Structured significant → orchestration vs depth.')
        if not np.isnan(ps[2]) and ps[2] < ALPHA:
            parts.append('Seq vs Par significant → parallelization effect.')
        if not np.isnan(ps[3]) and ps[3] < ALPHA:
            parts.append('Probabilistic vs Deterministic significant → conditional execution impact.')
    if not parts:
        parts.append('No significant structural contrast (or Kruskal–Wallis n.s.).')
    interpretations.append(f"{label}: " + " ".join(parts))

header = ' & '.join(['Metric'] + [escape_latex(c) for c in col_headers]) + ' \\\\'
body = '\n'.join(' & '.join([escape_latex(str(r[0]))] + list(r[1:])) + ' \\\\' for r in rows_tex)
tab = (
    '\\begin{table}[htbp]\n'
    '\\centering\n'
    '\\caption{Structural group contrasts: $p$-values (Holm) for Mann--Whitney $U$ tests. '
    'Bold: $p < 0.05$.}\n'
    '\\label{tab:structural_groups}\n'
    '\\begin{tabular}{lcccc}\n'
    '\\toprule\n' + header + '\n\\midrule\n' + body + '\n\\bottomrule\n'
    '\\end{tabular}\n\\end{table}\n'
)
# tab kept in memory for merged section; no individual .tex/.md/.csv written
csv_rows = []
for metric in metrics:
    if metric not in df.columns:
        continue
    label = METRIC_LABELS.get(metric, metric)
    kw_h, kw_p = kw_results.get(metric, (np.nan, np.nan))
    row = {'metric': label, 'KW_H': kw_h, 'KW_p': kw_p}
    for j, c in enumerate(['Dense_vs_Others', 'Centralized_vs_Structured', 'Seq_vs_Par', 'Prob_vs_Det']):
        row[c] = pairwise_holm.get(metric, [np.nan]*4)[j] if metric in pairwise_holm else np.nan
    csv_rows.append(row)
# pd.DataFrame(csv_rows).to_csv(...)  # only merged section written
pass
pd.DataFrame(rows_tex, columns=['Metric'] + col_headers)

,Metric,Dense vs Others,Centralized vs Structured,Seq vs Par,Probabilistic vs Deterministic
0,Throughput (RPS),\textbf{8.35e-11},0.092,0.673,0.121
1,Avg. response time (s),\textbf{9.03e-11},0.126,0.853,0.121
2,Energy (kJ),0.236,\textbf{0.024},0.569,\textbf{0.003}
3,CPU utilization,\textbf{1.10e-05},0.360,0.464,\textbf{5.90e-05}
4,Failure rate,\textbf{1.49e-13},\textbf{1.34e-09},\textbf{0.019},\textbf{0.014}
5,P95 latency (s),\textbf{2.59e-12},0.323,0.390,0.871


---

## Step 2 — RQ1: Energy differences across topologies

**2A.** Kruskal–Wallis on `energy`, `energy_per_request`, `energy_per_success`.  
**2B.** Structural contrasts (Mann–Whitney, Holm): Dense vs Others, Centralized vs Structured, Seq vs Par, Prob vs Det.  
Output: `energy_tests.tex`

In [8]:
ENERGY_METRICS = ['energy_kj']  # Only Energy (kJ) in paper; energy_per_request, energy_per_success excluded from output
# 2A Global KW
kw_energy = []
for m in ENERGY_METRICS:
    groups = [g[m].dropna().values for _, g in df.groupby('topology')]
    if any(len(g) < 2 for g in groups):
        continue
    h, p = stats.kruskal(*groups)
    kw_energy.append({'Metric': METRIC_LABELS.get(m, m.replace('_', ' ').title()), 'H': h, 'p': p})
kw_energy_df = pd.DataFrame(kw_energy)
print("2A. Kruskal–Wallis (energy metrics):")
print(kw_energy_df.to_string(index=False))

# 2B Structural contrasts (reuse pairwise_holm logic for these 3 metrics)
def run_four_contrasts(d, met):
    raw = [
        dense_vs_others(d, met), centralized_vs_structured(d, met),
        seq_vs_par(d, met), probabilistic_vs_deterministic(d, met)
    ]
    raw_f = [float(x) if not np.isnan(x) else 1.0 for x in raw]
    return holm_correction(raw_f)

energy_contrasts = []
for m in ENERGY_METRICS:
    h, p = stats.kruskal(*[g[m].dropna().values for _, g in df.groupby('structure_group')])
    holm = run_four_contrasts(df, m)
    cells = []
    for j, pv in enumerate(holm):
        s = f'{pv:.3f}' if pv >= 0.001 else f'{pv:.2e}'
        cells.append(f'\\textbf{{{s}}}' if pv < ALPHA else s)
    energy_contrasts.append([METRIC_LABELS.get(m, m.replace('_', ' ').title())] + cells)

cols = ['Dense vs Others', 'Centralized vs Structured', 'Seq vs Par', 'Prob vs Det']
energy_contrasts_df = pd.DataFrame(energy_contrasts, columns=['Metric'] + cols)
print("\n2B. Structural contrasts (p-values, bold if p<0.05):")
print(energy_contrasts_df.to_string(index=False))

2A. Kruskal–Wallis (energy metrics):
     Metric        H        p
Energy (kJ) 19.58097 0.001497

2B. Structural contrasts (p-values, bold if p<0.05):
     Metric Dense vs Others Centralized vs Structured Seq vs Par    Prob vs Det
Energy (kJ)           0.236            \textbf{0.024}      0.569 \textbf{0.003}


In [9]:
# Write energy_tests.tex (2A table + 2B table)
def esc(s):
    return str(s).replace('_', r'\_').replace('&', r'\&')
kw_lines = ' \\\\\n'.join([f"{esc(r['Metric'])} & {r['H']:.2f} & {r['p']:.3f}" + (' & \\textbf{*}' if r['p'] < ALPHA else '') for _, r in kw_energy_df.iterrows()])
ct_lines = ' \\\\\n'.join([' & '.join([esc(x) for x in r]) for r in energy_contrasts])
energy_tex = (
    "\\begin{table}[htbp]\n\\centering\n\\caption{RQ1: Energy across topologies. "
    "Top: Kruskal--Wallis. Bottom: Structural contrasts (Holm). Bold: $p<0.05$.}\n\\label{tab:energy_tests}\n"
    "\\small\n\\setlength{\\tabcolsep}{4pt}\n"
    "\\begin{tabular}{lcc}\n\\toprule\nMetric & H & $p$-value \\\\\n\\midrule\n" + kw_lines + " \\\\\n\\bottomrule\n\\end{tabular}\n\n"
    "\\begin{tabular}{lcccc}\n\\toprule\nMetric & Dense vs Others & Cent. vs Struct. & Seq vs Par & Prob vs Det \\\\\n\\midrule\n"
    + ct_lines + " \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n"
)
# only merged section written
pass

---

## Step 3 — Size interaction (differences increase with size?)

For **energy** and **throughput**: Topology effect, Size effect, Topology × Size interaction. Rank-transform then two-way comparison (Kruskal–Wallis per effect).

In [10]:
# Size interaction: Topology, Size, and simple interaction via KW per size
SIZE_METRICS = ['energy_kj', 'throughput_rps']
size_effect_rows = []
for m in SIZE_METRICS:
    if m not in df.columns:
        continue
    d = df[[m, 'topology', 'system_size']].dropna()
    # Topology effect (KW across topologies, all sizes)
    g_topo = [d[d['topology'] == t][m].values for t in d['topology'].unique()]
    h_t, p_t = stats.kruskal(*g_topo) if all(len(x) >= 2 for x in g_topo) else (np.nan, 1.0)
    # Size effect (KW across sizes, all topologies)
    g_size = [d[d['system_size'] == s][m].values for s in d['system_size'].unique()]
    h_s, p_s = stats.kruskal(*g_size) if all(len(x) >= 2 for x in g_size) else (np.nan, 1.0)
    # Interaction: KW(topology) at each size; report min p (pattern differs across sizes)
    p_int_list = []
    for sz in sorted(d['system_size'].unique()):
        sub = d[d['system_size'] == sz]
        g = [sub[sub['topology'] == t][m].values for t in sub['topology'].unique()]
        if all(len(x) >= 2 for x in g):
            _, p_int = stats.kruskal(*g)
            p_int_list.append(p_int)
    p_int = min(p_int_list) if p_int_list else np.nan
    lab = 'Energy (kJ)' if m == 'energy_kj' else 'Throughput (RPS)'
    size_effect_rows.append({'Metric': lab, 'Topology': p_t, 'Size': p_s, 'Interaction': p_int})
size_effect_df = pd.DataFrame(size_effect_rows)
print("Topology, Size, Interaction (p-values):")
print(size_effect_df.to_string(index=False))

Topology, Size, Interaction (p-values):
          Metric     Topology         Size  Interaction
     Energy (kJ) 1.497375e-03 4.268958e-28 8.940408e-11
Throughput (RPS) 3.984533e-10 1.014734e-15 1.007067e-10


In [11]:
def esc(s):
    return str(s).replace('_', r'\_').replace('&', r'\&')
def fmt_p(p):
    if np.isnan(p):
        return '—'
    s = f'{p:.3f}' if p >= 0.001 else f'{p:.2e}'
    return f'\\textbf{{{s}}}' if p < ALPHA else s
size_tex_rows = []
for _, r in size_effect_df.iterrows():
    row = [esc(r['Metric']), fmt_p(r['Topology']), fmt_p(r['Size']), fmt_p(r['Interaction'])]
    for i, k in enumerate(['Topology', 'Size', 'Interaction']):
        if r[k] < ALPHA and not np.isnan(r[k]):
            row[i+1] = f'\\textbf{{{row[i+1]}}}'
    size_tex_rows.append(' & '.join(row))
size_interaction_tex = (
    "\\begin{table}[htbp]\n\\centering\n\\caption{Topology $\\times$ Size: Kruskal--Wallis $p$-values. "
    "Interaction = min $p$ (topology effect) across sizes. Bold: $p<0.05$.}\n\\label{tab:size_interaction}\n"
    "\\small\n\\setlength{\\tabcolsep}{4pt}\n"
    "\\begin{tabular}{lccc}\n\\toprule\nMetric & Topology & Size & Interaction \\\\\n\\midrule\n"
    + ' \\\\\n'.join(size_tex_rows) + " \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n"
)
# only merged section written
pass

---

## Step 4 — RQ2: Performance–energy relationship

Per topology: Spearman correlation of throughput_rps, avg_latency_s, failure_rate with energy_kj. Output: `performance_energy_correlation.tex`

In [12]:
PERF_ENERGY_PAIRS = [
    ('throughput_rps', 'Throughput'),
    ('avg_latency_s', 'Latency'),
    ('failure_rate', 'Failure'),
]
corr_rows = []
for topo in df['topology'].unique():
    sub = df[df['topology'] == topo][['energy_kj', 'throughput_rps', 'avg_latency_s', 'failure_rate']].dropna()
    if len(sub) < 3:
        continue
    row = {'Topology': topo}
    for col, label in PERF_ENERGY_PAIRS:
        r, p = stats.spearmanr(sub['energy_kj'], sub[col])
        row[f'{label}_rho'] = r
        row[f'{label}_p'] = p
    corr_rows.append(row)
corr_df = pd.DataFrame(corr_rows)
print("Spearman (performance vs energy) per topology:")
print(corr_df.to_string(index=False))

Spearman (performance vs energy) per topology:
            Topology  Throughput_rho  Throughput_p  Latency_rho    Latency_p  Failure_rho    Failure_p
  probabilistic_tree       -0.381090  3.773106e-02     0.386429 3.491453e-02     0.340156 6.588236e-02
     parallel_fanout       -0.381980  3.724936e-02     0.378865 3.895723e-02     0.426473 1.876713e-02
chain_with_branching       -0.839377  6.781687e-09     0.838042 7.548697e-09     0.883871 9.638580e-11
        complex_mesh        0.956841  1.462528e-16    -0.959066 7.068084e-17    -0.722358 6.573768e-06
   hierarchical_tree       -0.844716  4.374279e-09     0.831368 1.271876e-08     0.797998 1.278193e-07
   sequential_fanout       -0.858065  1.355315e-09     0.853170 2.110565e-09    -0.639292 1.429442e-04


In [13]:
# Build performance_energy_correlation.tex: Topology | Throughput (rho, p) | Latency (rho, p) | Failure (rho, p)
def cell_rho_p(rho, p):
    s = f"{rho:.2f} ({p:.3f})" if p >= 0.001 else f"{rho:.2f} ({p:.2e})"
    if p < ALPHA:
        s = f"\\textbf{{{s}}}"
    return s
corr_tex_rows = []
for _, r in corr_df.iterrows():
    t = esc(r['Topology'].replace('_', ' ').title())
    c1 = cell_rho_p(r['Throughput_rho'], r['Throughput_p'])
    c2 = cell_rho_p(r['Latency_rho'], r['Latency_p'])
    c3 = cell_rho_p(r['Failure_rho'], r['Failure_p'])
    corr_tex_rows.append(f"{t} & {c1} & {c2} & {c3}")
corr_tex = (
    "\\begin{table}[htbp]\n\\centering\n\\caption{RQ2: Spearman correlation (performance vs energy) per topology. "
    "Bold: $p<0.05$.}\n\\label{tab:performance_energy_correlation}\n\\small\n\\setlength{\\tabcolsep}{4pt}\n"
    "\\begin{tabular}{lccc}\n\\toprule\nTopology & Throughput--Energy & Latency--Energy & Failure--Energy \\\\\n\\midrule\n"
    + ' \\\\\n'.join(corr_tex_rows) + " \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n"
)
# only merged section written
pass

---

## Step 5 — Efficiency: energy_per_rps

Kruskal–Wallis + structural contrasts for `energy_per_rps = energy / throughput_rps`. Enables statements like: *Probabilistic tree achieves statistically better energy efficiency than dense mesh.*

In [14]:
# energy_per_rps: KW + structural contrasts
m_eff = 'energy_per_rps'
g_topo = [df[df['topology'] == t][m_eff].dropna().values for t in df['topology'].unique()]
h_eff, p_eff = stats.kruskal(*g_topo) if all(len(x) >= 2 for x in g_topo) else (np.nan, 1.0)
kw_eff_row = {'Metric': 'Energy per RPS', 'H': h_eff, 'p': p_eff}
# Structural contrasts
holm_eff = run_four_contrasts(df, m_eff)
cells_eff = [f'{x:.3f}' if x >= 0.001 else f'{x:.2e}' for x in holm_eff]
for j, pv in enumerate(holm_eff):
    if pv < ALPHA:
        cells_eff[j] = f'\\textbf{{{cells_eff[j]}}}'
eff_contrast_row = ['Energy per RPS'] + cells_eff
print("Energy per RPS: H =", round(h_eff, 2), ", p =", round(p_eff, 4))
print("Structural contrasts:", eff_contrast_row)
# Append to energy tables and re-save (optional: or save as efficiency.tex)
kw_energy_df_plus = pd.concat([kw_energy_df, pd.DataFrame([kw_eff_row])], ignore_index=True)
energy_contrasts_plus = energy_contrasts + [eff_contrast_row]
eff_tex = (
    "\\begin{table}[htbp]\n\\centering\n\\caption{Energy efficiency (energy per RPS): Kruskal--Wallis and structural contrasts. Bold: $p<0.05$.}\n"
    "\\label{tab:efficiency}\n\\small\n\\setlength{\\tabcolsep}{4pt}\n"
    "\\begin{tabular}{lcc}\n\\toprule\nMetric & H & $p$-value \\\\\n\\midrule\n"
    f"Energy per RPS & {h_eff:.2f} & {p_eff:.3f}" + (" & \\textbf{*}" if p_eff < ALPHA else "") + " \\\\\n\\bottomrule\n\\end{tabular}\n\n"
    "\\begin{tabular}{lcccc}\n\\toprule\nMetric & Dense vs Others & Cent. vs Struct. & Seq vs Par & Prob vs Det \\\\\n\\midrule\n"
    + ' & '.join([esc(x) for x in eff_contrast_row]) + " \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n"
)
# only merged section written
pass

Energy per RPS: H = 39.27 , p = 0.0
Structural contrasts: ['Energy per RPS', '\\textbf{6.46e-05}', '0.213', '0.530', '\\textbf{8.08e-05}']


---

## Merged LaTeX section

Single section with `\small` and `\setlength{\tabcolsep}{4pt}` for the paper. No effect sizes; p-values only; bold if \(p < 0.05\).

In [15]:
# Merged section: one .tex file to include in the paper
merged_content = (
    "% Statistical analysis section for paper. Use \\small and compact column spacing.\n"
    "\\small\n\\setlength{\\tabcolsep}{4pt}\n\n"
    "% Normality (documentation)\n"
    + norm_paragraph + "\n\n"
    "% RQ1 & structural groups\n"
    + tab + "\n\n"
    "% RQ1 Energy\n"
    + energy_tex + "\n\n"
    "% Size interaction\n"
    + size_interaction_tex + "\n\n"
    "% RQ2 Performance--energy correlation\n"
    + corr_tex
)
TABLES_DIR.joinpath('table3_4_5_statistics.tex').write_text(merged_content)
print("Saved tables/table3_4_5_statistics.tex (include in paper)")

Saved tables/table3_4_5_statistics.tex (include in paper)


In [16]:
# Show interpretation in notebook
from IPython.display import Markdown, display
display(Markdown("# Interpretation (structural group contrasts)\n\n" + "\n\n".join(interpretations)))

# Interpretation (structural group contrasts)

Throughput (RPS): Dense vs Others significant → dense coordination cost.

Avg. response time (s): Dense vs Others significant → dense coordination cost.

Energy (kJ): Centralized vs Structured significant → orchestration vs depth. Probabilistic vs Deterministic significant → conditional execution impact.

CPU utilization: Dense vs Others significant → dense coordination cost. Probabilistic vs Deterministic significant → conditional execution impact.

Failure rate: Dense vs Others significant → dense coordination cost. Centralized vs Structured significant → orchestration vs depth. Seq vs Par significant → parallelization effect. Probabilistic vs Deterministic significant → conditional execution impact.

P95 latency (s): Dense vs Others significant → dense coordination cost.